# Agregar un control deslizante al censo de exoplanetas

En la última parte de este episodio, terminaremos nuestra copia del gráfico del _Censo de exoplanetas_ que se puede encontrar en [Sitio de la NASA sobre exoplanetas](https://exoplanets.nasa.gov/discovery/discoveries-dashboard/):

<img src="../img/Exoplanet Census.png" alt="Exoplanet Census" width="400"/>

Aprendimos cómo procesar los datos con la biblioteca `FSharp.Data` y cómo gráficarlos con `Plotly`, y ahora necesitamos agregar un control deslizante al gráfico, para que podamos ver qué exoplanetas se descubrieron hasta un momento determinado. año.

Comencemos abriendo todas las bibliotecas y leyendo y procesando nuestros datos:

In [1]:
#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"
#r "nuget: FSharp.Data"

Installed Packages FSharp.Data, 6.4.0 Plotly.NET, 4.2.0 Plotly.NET.Interactive, 4.2.0

Loading extensions from `/Users/flavioc/.nuget/packages/plotly.net.interactive/4.2.0/lib/netstandard2.1/Plotly.NET.Interactive.dll`

In [2]:
open FSharp.Data
open Plotly.NET

In [3]:
[<Literal>]
let exoplanetsFile = "../data/consolidatedExoplanets.csv"

type ExoPlanetType = FSharp.Data.CsvProvider<exoplanetsFile, HasHeaders=true, PreferOptionals=true>
let planets = ExoPlanetType.GetSample()

Definimos una función para obtener el año del descubrimiento.

In [4]:
let getYearOfDiscovery (ref: string) = 
    let found = RegularExpressions.Regex.Matches(ref, " \d{4}")
    match found.Count with
    | 0 -> None
    | 1 -> Some (found.[0].Value.Trim(' ') |> int) 
    | _ -> failwith "More than one year found"
    

y luego un tipo útil para los datos

In [5]:
type ExoCensusData =
    {
        DiscoveryMethodName : string 
        OrbitTimes: decimal 
        Masses: decimal  
        YearOfDiscovery: int 
    }

Y finalmente se crea una secuencia de los mismos. Usamos una tupla para guardar los datos intermedios de la tabla, ya que hay algunos valores de `Option`, y luego los filtramos a nuestro `ExoCensusData`:

In [6]:
let data =
    planets.Rows 
    |> Seq.map (fun row -> (row.Discoverymethod, row.Pl_orbper, row.Pl_masse, getYearOfDiscovery row.Disc_refname))
    |> Seq.choose (fun (method, period, mass, year) -> 
        match period,mass,year with
        | Some p, Some m, Some y -> Some {DiscoveryMethodName = method; OrbitTimes = p; Masses = m; YearOfDiscovery = y}
        | _ -> None)

In [7]:
data.DisplayTable()

DiscoveryMethodName,OrbitTimes,Masses,YearOfDiscovery
Transit,3.1013125714285714285714285714,222.47977142857142857142857143,2005
Imaging,73275.446666666666666666666667,3000.00000,2008
Transit,12.33334285,271.05000,2012
Transit,6.02978806,3.78000,2014
Transit,11.578912324,19.80000,2014
Transit,10.558297226,14.70000,2014
Radial Velocity,3185.41500000,7838.32346,2012
Transit,3.89905200,271.74465,2020
Transit,3.1186028214285714285714285714,297.603516,2009
Transit,2.1751799028571428571428571429,292.1623025,2012


Agrupamos nuestros datos por método de descubrimiento.

In [8]:
let dataByDiscoveryMethod = 
    data 
    |> Seq.groupBy (fun exoData -> exoData.DiscoveryMethodName)

y luego creó el tipo de datos de la figura:

In [9]:
type ExoTrace = 
    {
        DiscoveryMethodName : string 
        OrbitTimes : seq<decimal>
        Masses : seq<decimal>
        }


In [10]:
let exoTraces = 
    dataByDiscoveryMethod
    |> Seq.map (fun (method, data) -> 
            let orbits = data |> Seq.map (fun exoData -> exoData.OrbitTimes)
            let masses = data |> Seq.map (fun exoData -> exoData.Masses)
            {DiscoveryMethodName = method; OrbitTimes = orbits; Masses = masses})

También definimos varios valores para personalizar el diseño y la configuración de la figura:

In [11]:
open Plotly.NET.LayoutObjects // this namespace contains all object abstractions for layout styling

let orbPeriodAxis =
    LinearAxis.init (
        Title = Title.init (Text = "ORBIT PERIOD (EARTH DAYS)"),
        AxisType = StyleParam.AxisType.Log,
        ShowLine = true,
        ShowGrid = false,
        Range = StyleParam.Range.MinMax (-2, 8),
        Ticks = StyleParam.TickOptions.Outside
    )

let massLogAxis =
    LinearAxis.init (
        Title = Title.init (Text = "PLANET MASS (EARTH MASSES)"),
        AxisType = StyleParam.AxisType.Log,
        ShowLine = true,
        ShowGrid = false,
        Ticks = StyleParam.TickOptions.Outside
    )

let openCircle = 
    StyleParam.MarkerSymbol.Modified(
            StyleParam.MarkerSymbol.Circle,
            StyleParam.SymbolStyle.Open
        )  

In [12]:
open Plotly.NET.ConfigObjects

let layout =
    Layout.init(
                Width = 1000,
                Height = 500
    )


Finalmente, obtenemos la figura para todos los datos:

In [13]:
exoTraces
|> Seq.map (fun exo -> 
                Chart.Point(exo.OrbitTimes,exo.Masses, Name = exo.DiscoveryMethodName)
                |> Chart.withMarkerStyle(Symbol=openCircle))
|> Chart.combine
|> Chart.withXAxis orbPeriodAxis
|> Chart.withYAxis massLogAxis
|> Chart.withLayout layout 

<!-- Plotly chart will be drawn inside this DIV -->

### Agregar el control deslizante del año

Una de las posibilidades que nos brinda `Plotly` es agregar un control deslizante para cambiar la figura de forma interactiva. El gráfico del censo de exoplanetas tiene un control deslizante que selecciona el número de planetas según el año del descubrimiento.
Pensemos en este tipo por un minuto. Estamos trabajando en un gráfico que mostrará los exoplanetas descubiertos hasta un año determinado. Por lo tanto, debemos poder filtrar los datos de acuerdo con esa condición _antes_ de construir los rastros para graficar.

Por tanto, debemos poder filtrar los datos por año de descubrimiento. Creemos el rango de años:

In [14]:
let yearsOfDiscovery = [1989..1..2023]
yearsOfDiscovery

[ 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008 ... (more) ] HeadOrDefault 1989 TailOrNull [ 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009 ... (more) ] HeadOrDefault 1990 TailOrNull [ 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010 ... (more) ] HeadOrDefault 1991 TailOrNull [ 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011 ... (more) ] HeadOrDefault 1992 TailOrNull [ 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012 ... (more) ] HeadOrDefault 1993 TailOrNull [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] HeadOrDefault 1994 TailOrNull [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] Head 1994 Tail [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] (values) [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] Head 1993 Tail [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] HeadOrDefault 1994 TailOrNull [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] Head 1994 Tail [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] (values) [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] (values) [ 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012 ... (more) ] Head 1992 Tail [ 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012 ... (more) ] HeadOrDefault 1993 TailOrNull [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] HeadOrDefault 1994 TailOrNull [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] Head 1994 Tail [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] (values) [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] Head 1993 Tail [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] HeadOrDefault 1994 TailOrNull [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] Head 1994 Tail [ 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014 ... (more) ] (values) [ 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013 ... (more) ] (values) [ 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012 ... (more) ] (values) [ 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011 ... (more) ] Head 1991 Tail [ 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011 ... (more) ] HeadOrDefault 1992 TailOrNull [ 1

Ahora calculamos los datos para cada `Trace`. Definimos una función `exoTracesUpToYear` que recibe un año y crea los datos correspondientes para las gráficas.

In [15]:
let exoTracesUpToYear year =
    dataByDiscoveryMethod
    |> Seq.map (fun (method, data) -> 
            let dataUpToYear = 
                data 
                |> Seq.filter (fun exoData -> exoData.YearOfDiscovery <= year)

            let orbits = 
                dataUpToYear
                |> Seq.map (fun exoData -> exoData.OrbitTimes)
            let masses = 
                dataUpToYear 
                |> Seq.map (fun exoData -> exoData.Masses)
            {DiscoveryMethodName = method; OrbitTimes = orbits; Masses = masses}
            )
    |> Seq.sortBy (fun exo -> exo.DiscoveryMethodName)

¿Cuántas graficas tenemos para cada año?

In [31]:
yearsOfDiscovery
    |> Seq.map (fun year -> year,exoTracesUpToYear year |> Seq.length)
    |> Seq.iter (fun (year,count) -> printfn "%d,%d" year count)

1989,9
1990,9
1991,9
1992,9
1993,9
1994,9
1995,9
1996,9
1997,9
1998,9
1999,9
2000,9
2001,9
2002,9
2003,9
2004,9
2005,9
2006,9
2007,9
2008,9
2009,9
2010,9
2011,9
2012,9
2013,9
2014,9
2015,9
2016,9
2017,9
2018,9
2019,9
2020,9
2021,9
2022,9
2023,9


La forma en que estamos construyendo nuestro conjunto de gráficas define una secuencia vacía `seq []` para los métodos de descubrimiento que no estaban disponibles en un año determinado. Por lo tanto, tenemos nueve gráficas para cada dato hasta un año dado. Definamos un valor que será útil en breve:

In [20]:
let numberOfDiscoveryMethods = 9 

Al inspeccionar el [ejemplo de control deslizante](https://plotly.net/chart-layout/sliders.html), observamos que hay que hacer una figura con todas las gráficas posibles que acomodará el control deslizante, y luego hacer visible aquellas para el año seleccionado en el control deslizante, es decir, debemos seleccionar solo las gráficas para un año determinado.

Para conectar una figura con el control deslizante, necesitamos crear un objeto `Slider` a través de la función `Slider.init` de Plotly. Uno de los argumentos que recibe este constructor es `Steps`, que es una `seq` de objetos `SliderStep`. Este objeto lleva un argumento `Method` que describe cómo actualizar el gráfico cuando se mueve el control deslizante, un `Label` o etiqueta para imprimir el valor actual del control deslizante y `Args` que son los argumentos que se pasan al método `Method`. Uno de los argumentos es `visible`, que determina qué trazos son visibles para cada posición del control deslizante.

> Quizás esto no parece demasiado al estilo típico funcional de F#, pero tenga en cuenta que `Plotly` es originalmente una biblioteca de JavaScript y la especificación de cada gráfico es un archivo JSON.

> Esta [respuesta](https://community.plotly.com/t/multiple-traces-with-a-single-slider-in-plotly/16356/2) (aunque en Python) es extremadamente útil para comprender cómo funciona el control deslizante para múltiples trazos.

Para que funcione correctamente, hay que crear un gráfico con _todos_ los trazos que se quieren mostrar y seleccionarlos en consecuencia con el argumento `visible`. Luego, necesitamos dibujar nueve gráficas para cada año, para todos los años desde 1989 (35), para un total de 9 x 35 = 315 trazos. Como para cada año tenemos nueve gráficas para mostrar, `visible` será una máscara (implementada como una secuencia de booleanos), siendo falsa para todas las trazas, excepto las nueve que se mostrarán para un año determinado.

Comencemos creando una matriz de valores booleanos `numberOfDiscoveryMethods * (yearsOfDiscovery |> Seq.length)` establecidos en `false`:

In [32]:
let visibleInit = 
    Array.create (numberOfDiscoveryMethods * (yearsOfDiscovery |> Seq.length)) false    

In [22]:
visibleInit |> Seq.length // 9 (métodos de descubrimiento) x 35 (años desde 1989 a 2023 incluído)= 315

315

y se crea una función que devuelva la máscara `visible` como una secuencia hasta un año determinado:

In [33]:
let setVisibleForYear year = 
    let i0 = year - 1989 
    visibleInit
    |> Seq.mapi (fun i visible -> 
                    if i >= (i0 * numberOfDiscoveryMethods) && 
                       i <  ((i0 + 1) * numberOfDiscoveryMethods) then true
                    else false 
                )
    |> Array.ofSeq
                    

Así, por ejemplo, para 1989, la máscara comenzará con nueve `true`s consecutivos, seguidos de todos los `false`s:

In [34]:
let visibility1989 = setVisibleForYear 1989
visibility1989.DisplayTable()

value
True
True
True
True
True
True
True
True
True
False


Para 1990, la máscara comenzará con nueve `false`s consecutivos, seguidos de nueve `true`s y continuará con todos los `false`s:

In [35]:
let visibility1990 = setVisibleForYear 1990
visibility1990.DisplayTable()

value
False
False
False
False
False
False
False
False
False
True


Etcétera. Ahora podemos construir el arreglo `SliderStep`, siguiendo el ejemplo de la documentación de `Plotly`:

In [36]:
let sliderSteps =
    yearsOfDiscovery
    |> Seq.indexed
    |> Seq.map (fun (i, year) ->
        // Create a visibility and a title parameter
        // The visibility parameter includes an array where every parameter
        // is mapped onto the trace visibility
        let visible = setVisibleForYear year |> box  // box convierte un tipo en un objeto genérico
        let title = sprintf "Year: %d" year |> box

        SliderStep.init (
            Args = [ "visible", visible; "title", title ],
            Method = StyleParam.Method.Update,
            Label = string (year)
        ))
        

Una vez que tengamos esta secuencia, podemos construir el objeto `Slider`:

In [27]:
let slider =
    Slider.init (
        CurrentValue = SliderCurrentValue.init (Prefix = "Year: "),
        Padding = Padding.init (T = 50),
        Steps = sliderSteps
    )        

¡Ya casi llegamos! Ahora recorremos todos los valores posibles del control deslizante (está indexado de 0 a `yearsOfDiscovery.Length - 1`), creamos los gráficos para cada conjunto de datos y les damos formato a todos con nuestra elección de eje y diseño como antes:

In [28]:
let exoCensusChart =
    Seq.init yearsOfDiscovery.Length (fun i -> i)
    |> Seq.map (fun yearIdx ->        
        // Some plot must be visible here or the chart is empty at the beginning
        let chartVisibility = 
            if yearIdx = 0 then
                StyleParam.Visible.True
            else
                StyleParam.Visible.False

        let go =
            (yearIdx + 1989)
            |> exoTracesUpToYear
            |> Seq.map (fun exo -> 
                            Chart.Point(exo.OrbitTimes,exo.Masses, Name = exo.DiscoveryMethodName)
                            |> Chart.withMarkerStyle(Symbol=openCircle)
                            |> Chart.withTraceInfo (Visible = chartVisibility))
            |> Chart.combine            

        go) 
    |> GenericChart.combine
    |> Chart.withXAxis orbPeriodAxis
    |> Chart.withYAxis massLogAxis
    |> Chart.withLayout layout 

Como nota al margen, usamos el valor `chartVisibility` para borrar el gráfico de 1989. Ahora, pasamos el `scatterChart` y el `Slider` a `Chart.withSlider` para crear el gráfico completo:

In [29]:
let chart = exoCensusChart |> Chart.withSlider slider

In [37]:
chart

<!-- Plotly chart will be drawn inside this DIV -->

¡Y voilá! ¡Nuestro hermoso censo de exoplanetas! ¡Mueva el control deslizante para ver aparecer los planetas descubiertos hasta un determinado año!

## Terminando

Finalmente, nuestro ejemplo está completo, pasamos del procesamiento de datos con proveedores de tipos a nuestro gráfico de censo de exoplanetas. Esto muestra un flujo de trabajo típico para el procesamiento de datos en F#. En el camino, revisamos cómo usar los poderosos proveedores de tipos para administrar información estructurada, e incluso los usamos para crear y guardar datos manipulados en un archivo.

`Plotly` es una biblioteca de gráficos fantástica, diseñada para la Web. De hecho, se puede crear un html con:


In [38]:
chart |> Chart.saveHtml "exoplanets census"

y abrirlo con un navegador.